# Module 3  -  Exercises

The exercises in this are partially complete. The data used in this notebook is taken from [kaggle website](). The data is about Climate Change, Earth Surface Temperature. We will load the data into reshift cluster and query the data. We will generate visualizations based on the data extracted from the redshift database table. 


**Note: ** The questions are highlighted in red. 

In [ ]:
import boto3
import random
import time
import json
import psycopg2
from getpass import getpass
from pandas import read_sql
import datetime

redshift_client = boto3.client('redshift')

# Give a password to your redshift cluster
pwd = getpass('password')

Set the names of the security group for the cluster, names of the cluster and database itself. 

In [ ]:
Sec_group_name= "climate_sec_group"
cluster_name="climate"
database_name="climatecitydata"

Create an AWS EC2 client object to create a security group for the redshift cluster. We are going to deploy the cluster with default parameters already set using aws configure command. 

If you still haven't gone through AWS command line interface tools, we strongly recommend you to do it by clikcing the link below.

[click here to open the lab Accesing_AWS_through_CLI](../../module2/extra_labs/Accessing_AWS_through_CLI.ipynb)

In [ ]:
ec2_client = boto3.client('ec2')

# The function will take the default credentials stored in configure file

<br>
<span style="color:red"><b>Activity 1:</b> Create a security group named "redshift_Sec_group".</span>


In [ ]:
sg = ec2_client.create_security_group(
    
    ## you code for activity 1 goes here
    
)

Edit the security group inbound rules to allow all TCP/IP traffic on port number 5439. 

In [ ]:
try:
    sec_rule="ALL TCP"
    data = ec2_client.authorize_security_group_ingress(
        GroupId=Sec_group,
        IpPermissions=[
            {'IpProtocol': 'tcp',
             'FromPort': 5439,
             
             'ToPort': 5439,
             'IpRanges': [{'CidrIp': '0.0.0.0/0'}]},
        ],)
    print("Ingress "+sec_rule+" added")
except:
    print(sec_rule+" already added")
#     print(data)


### Create a keypair

Create a keypair for the EC2 instance. We first generate a name to create a key with that name and also store the key in a file. ec2.create_key_pair() will create a keypair. System command echo is used to write the contents of keypair generated to a file created with same name as keypair. 

You have to modify the file permissions to provide readonly access. If the file is open, system will throw an error. Do chmod(file, 0o400) 

In [ ]:
import time 
import os
import getpass
from subprocess import call

#Set the username from system
system_user_name=getpass.getuser()

ec2_pem_file=time.strftime("EC2-%d%m%Y%H%M%S-"+system_user_name)
ec2_key=ec2_client.create_key_pair(KeyName=ec2_pem_file)

#Don't do this unless you have a good reason
#print(emr_key['KeyMaterial'])

os.system("echo \""+ec2_key['KeyMaterial']+"\" > "+ec2_pem_file+".pem")
os.chmod(ec2_pem_file+".pem",0o400)

print("KeyName         : "+ec2_key['KeyName']+"\nKey Fingerprint : "+ec2_key['KeyFingerprint'])

<br>
<span style="color:red"><b>Activity 2:</b> Deploy a redshift cluster with default database as "climatecitydata" stored in the variable "database_name". The cluster should have 2 slave nodes. Use the security group created above for creating the cluster.</span>


In [ ]:
response = redshift_client.create_cluster(
    
    ## you code for activity 2 goes here


)

Once the cluster is created use the below poll function to check the status of the cluster. Once it is in ready state the poll function will indicate the cluster is ready for use.

In [ ]:
def poll_until_completed(client, cluster_id):
    delay = 2
    while True:
        cluster = client.describe_clusters(ClusterIdentifier=cluster_id)
#         for tag in cluster:
#             print(tag)
#         print(cluster)
#         print(cluster['Clusters'][0]['ClusterIdentifier'])
        status = cluster['Clusters'][0]['ClusterStatus']
#         message = cluster.get('Message', '')
        now = str(datetime.datetime.now().time())
        print("cluster %s is %s at %s" % (cluster_id, status, now))
        if status in ['available', 'final-snapshot']:
            break

        # exponential backoff with jitter
        delay *= random.uniform(1.1, 2.0)
        time.sleep(delay)

In [ ]:
poll_until_completed(redshift_client, cluster_id=cluster_name)  # Can't use the cluster until it is available

In [ ]:
# Run the cell if you want to see the complete details of cluster. 

# redshift_client.describe_clusters()

To connect to the cluster we need its endpoint. Below cell prints the end point, the default port where the cluster is listening for input requests and the database available in the cluster. 

In [ ]:
cluster_end_point = ''
for cluster in redshift_client.describe_clusters()["Clusters"]:
    print("Cluster endpoint:",str(cluster["Endpoint"]["Address"])+"\n"+"Port:",str(cluster["Endpoint"]["Port"])+"\n"+"Database:",str(cluster["DBName"]))
    cluster_end_point = str(cluster["Endpoint"]["Address"])

Below code cell prints the public and private addresses of the nodes in cluster. 

In [ ]:
for cluster in redshift_client.describe_clusters()["Clusters"]:
    for ClusterNode in cluster["ClusterNodes"]:
        if cluster_name in cluster["Endpoint"]["Address"]:
            print(ClusterNode)

<br>
<span style="color:red"><b>Activity 3:</b> Create a connection string to connect to redshift cluster created above</span>


In [ ]:
conn_string = { 
    
        ## you code for activity 3 goes here

    
         }

Once the connection string is available, use the create_conn() method to create a connection object to connect to "climatecitydata" database.

In [ ]:
def create_conn(config):
    try:
        con=psycopg2.connect(dbname=config['dbname'], host=config['host'], 
                              port=config['port'], user=config['user'], 
                              password=config['pwd'])
        return con
    except Exception as err:
        print(err)

In [ ]:
con = create_conn(config=conn_string)

We have established the connection to redshift cluster using psycopg library. 

In [ ]:
con

## Load the data into Redshift cluster

The data is already uploaded to S3 bucket 'skaf48bucket00' and should be publicly accesible. So create a redshift table and copy the data from S3 into the table. 

Create a table called "dsaclimatecitydata" using below statement.

In [ ]:
statement="""create table dsaclimatecitydata (dt date,averagetemperature numeric(10,5),
            averagetemperatureuncertainty numeric(10,5),city varchar(25),country varchar(34),
            latitude varchar(6),longitude varchar(7));"""

In [ ]:
# Create a cursor object and execute above statement to create the table.
cur = con.cursor()

In [ ]:
cur.execute(statement)
con.commit()

In [ ]:
# con.rollback()

<br>
<span style="color:red">Note: </span>

    Update the blanks for access id and secret key in below cell. Run the updated cell to copy data from S3 file to the database table. 

In [ ]:
sql = """copy dsaclimatecitydata from 's3://"""+bucket_name+"""/GlobalLandTemperaturesByCity.csv'
    access_key_id '<give your credentials>'
    secret_access_key '<give your credentials>'
    region 'us-east-1'
    ignoreheader 1
    null as 'NA'
    removequotes
    delimiter ',';"""
cur.execute(sql)
con.commit()

In [ ]:
# Uncomment below lines and run the table if the database throws any error. The stl_load_errors table captures the error 
# messages. 

# df=read_sql("select *from stl_load_errors",con)
# df

In [ ]:
df = read_sql("select * from dsaclimatecitydata limit 10;",con=con)
df

Lets create a table and load the data into Redshift. We established connection to the cluster above. Use the connection object "con" to execute create table srtatement.  

In [ ]:
df = read_sql("select column_name, data_type, character_maximum_length \
from INFORMATION_SCHEMA.COLUMNS where table_name = 'dsaclimatecitydata';",con=con)
df

We will stage the data on S3 first before writing it to redshift cluster. 

In [ ]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [ ]:
df = read_sql("""select * from dsaclimatecitydata where city = 'Hyderabad' limit 5;""",con)
df

<br>
<span style="color:red"><b>Activity 4:</b> Create a new table called dsaclimatesubsetdata1 with columns dt, averagetemperature, month, year, city, country, latitude, longitude and the column dt as primary key. Use select query on dsaclimatecitydata table to create the table. 

Hint: The columns month, year are created by extracting month and year from dt column. 

</span>

Sample table output: 


```

	dt	averagetemperature	month	year	city	country	latitude	longitude
0	1743-11-01	5.339	11.0	1743.0	Alexandria	United States	39.38N	76.99W
1	1743-11-01	7.431	11.0	1743.0	Almere	Netherlands	52.24N	5.26E
2	1743-11-01	-5.556	11.0	1743.0	Almetyevsk	Russia	55.45N	51.02E
3	1743-11-01	7.431	11.0	1743.0	Amersfoort	Netherlands	52.24N	5.26E
4	1743-11-01	3.510	11.0	1743.0	Armavir	Russia	45.81N	40.38E

```

In [ ]:
statement = """create table dsaclimatesubsetdata1 ....... """

In [ ]:
con.rollback()

In [ ]:
cur.execute(statement)
con.commit()

In [ ]:
df = read_sql("""select * from dsaclimatesubsetdata1 limit 5;""",con)
df

In [ ]:
df = read_sql("select column_name, data_type, character_maximum_length \
from INFORMATION_SCHEMA.COLUMNS where table_name = 'dsaclimatesubsetdata1';",con=con)
df

<br>
<span style="color:red"><b>Activity 5:</b> Select median of averagetemperature as column named median and year from the table dsaclimatesubsetdata. Group the data by year. Order the data in ascending of year.  

</span>


Sample output

```

	median	year
0	5.354	1743.0
1	11.552	1744.0
2	2.196	1745.0
3	0.000	1746.0
4	0.000	1747.0

```

In [ ]:
## Fill the empty quotes to answer activity 5

df = read_sql("""  """,con)
df.head()

<br>
<span style="color:red"><b>Activity 6:</b>  Use the data in variable df in above cell to plot year on x-axis and median on y-axis. 

</span>


In [ ]:
import matplotlib.pyplot as plt

# Plot the data
plt.plot( ## your code for activity 6 goes here )

# Add a legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
con.rollback()

<br>
<span style="color:red"><b>Activity 7:</b> Select median of averagetemperature as column named median and year from the table dsaclimatesubsetdata. Group the data by year. Order the data in ascending of year and year is >=1900.  

PLot the year on x-axis and median on y-axis once the query results are obtained

</span>


Sample output

```

median	year
0	20.0940	1900.0
1	19.9100	1901.0
2	19.8990	1902.0
3	19.5085	1903.0
4	19.5590	1904.0

```

In [ ]:
## Fill the empty quotes to answer activity 7

df = read_sql("""   """,con)
df.head()

In [ ]:
import matplotlib.pyplot as plt

# Plot the data
plt.plot( ## your code for activity 7 goes here )

# Add a legend
plt.legend()

# Show the plot
plt.show()

In [ ]:
import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls


trace=go.Scatter(
    x=df['year'],
    y=df['median'],
    mode='lines',
    )
data=[trace]

py.iplot(data, filename='line-mode')

In [ ]:
con.rollback()

<br>
<span style="color:red"><b>Activity 8:</b> Select median of averagetemperature as column named median, year, country from the table dsaclimatesubsetdata where country is among ['United States','China','India','Japan','Germany','United Kingdom']. Group the data by year. Order the data in ascending of year and year is >=1950.  


</span>


Sample output

```

	median	year	country
0	26.163995	1950.0	India
1	15.168995	1950.0	China
2	8.500000	1950.0	United Kingdom
3	7.484000	1950.0	Germany
4	13.644000	1950.0	Japan
```

In [ ]:
## Fill the empty quotes to answer activity 8

df = read_sql("""   """,con)
df.head()

In [ ]:
abc=df.pivot('year','country','median')
f,ax=plt.subplots(figsize=(20,10))
abc.plot(ax=ax)

In [ ]:
con.rollback()

<br>
<span style="color:red"><b>Activity 9:</b> Select max of averagetemperature as column named max_temp, country from the table dsaclimatesubsetdata. Group the data by country. Order the data by descending order of max_temp. Limit the results to count 20.


Use a barplot as shown in labs and plot the temperatures on y-axis and countries on x-axis.

</span>


Sample output

```

	max_temp	country
0	39.651	Algeria
1	39.156	Iran
2	38.283	Iraq
3	38.049	Saudi Arabia
4	37.938	Pakistan

```

In [ ]:
## Fill the empty quotes to answer activity 9

df = read_sql("""    """,con)
df.head()

In [ ]:
## Complete the code to answer activity 9


import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


bar_plot = 

## Delete the cluster

In [ ]:
# response = redshift_client.delete_cluster(
#     ClusterIdentifier='climate',
#     SkipFinalClusterSnapshot=True
# )

response = redshift_client.delete_cluster(
    ClusterIdentifier='climate',
    SkipFinalClusterSnapshot=True
)